In [ ]:
list.of.packages <- c("tidyverse", "data.table", "dtplyr")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)

library(tidyverse)
library(data.table)
library(dtplyr)
library(dplyr, warn.conflicts = FALSE)

In [ ]:
raw_shots <- read_csv("../Data/NBA_Shots_Raw.csv")
player_info <- read_csv("../Data/archive/csv/common_player_info.csv")
player_info <- player_info %>% rename(PLAYER_NAME="display_first_last")

In [ ]:
colnames(raw_shots)
colnames(player_info)
unique(player_info$position)
head(raw_shots)

In [ ]:
s <- raw_shots %>% 
    left_join(player_info %>% 
    select(position, PLAYER_NAME), by="PLAYER_NAME") %>% 
    filter(position == "Guard") %>% 
    select(SHOT_DIST)

In [ ]:
ggplot(s, aes(x=SHOT_DIST)) + 
    geom_histogram(bins=30)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)

s <- raw_shots %>% 
    left_join(player_info %>% select(position, PLAYER_NAME), by = "PLAYER_NAME") %>%
    filter(!is.na(position)) %>%
    select(SHOT_DIST, position)

ggplot(s, aes(x = SHOT_DIST)) +
    geom_histogram(bins = 30, fill = "blue", color = "black", alpha = 0.5) +
    facet_wrap(~ position) +
    labs(title = "Shot Distance Distribution by Position",
       x = "Shot Distance (feet)",
       y = "Frequency") +
    theme_minimal()

In [ ]:
raw_shots %>%
    left_join(player_info %>% select(position, PLAYER_NAME), by = "PLAYER_NAME", relationship="many-to-many") %>%
    filter(!is.na(position)) %>%
    group_by(PLAYER_NAME, position) %>% 
    summarize(tot_FGM = sum(FGM), .groups="drop") %>% 
    arrange(desc(tot_FGM))

In [ ]:
shooter_data <- raw_shots %>% 
    filter(FGM == 1) %>% 
    group_by(PLAYER_NAME) %>%
    summarise(
        total_FGM = n(),
        pct_3pt = sum(PTS_TYPE == 3),
        pct_2pt = sum(PTS_TYPE == 2)
    ) %>%
    left_join(player_info %>% select(position, PLAYER_NAME, team_abbreviation), by = "PLAYER_NAME", relationship="many-to-many")

In [ ]:
shooter_data %>%
    group_by(team_abbreviation) %>%
    #slice_max(order_by=pct_3pt, n=1) %>%
    arrange(desc(total_FGM))